In [5]:
# Kaggle setup
import sys
sys.path.append('/kaggle/working/python_packages')

# Install packages (run only once)
!pip install numpy==1.26.4 opencv-python ultralytics -t /kaggle/working/python_packages


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 9.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 88.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 31.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 109.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import sys
sys.path.append('/kaggle/working/python_packages')

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
import shutil

print("NumPy:", np.__version__)
print("OpenCV:", cv2.__version__)
print("✅ Ultralytics: YOLO imported successfully!")  # ultralytics.__version__ doesn't exist



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
NumPy: 2.0.2
OpenCV: 4.12.0
✅ Ultralytics: YOLO imported successfully!


In [9]:
# Verify GPU
import os
import torch

# ================== GPU CHECK ==================
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU count: {torch.cuda.device_count()}")

# ================== PATHS ==================
VIDEO_PATH = "/kaggle/input/exam-videos/Sus_head_1_45c.mp4"
OUTPUT_DIR = "/kaggle/working/output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ================== CONFIG ==================
IMG_SIZE = 640
ROI_SIZE = 224
FRAME_SKIP = 10
KPT_CONF_TH = 0.5

JOINT_RADIUS = 3
BODY_BONE_THICKNESS = 3
HEAD_BONE_THICKNESS = 2   # <<< THIS IS THE FIX

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✅ Setup complete. Output dir: {OUTPUT_DIR}")

GPU available: True
GPU count: 0
✅ Setup complete. Output dir: /kaggle/working/output


In [8]:
 output runs skeleton_images.zip yolo-skeleton-results.zip
!rm -rf  /kaggle/working/runs
!rm -rf /kaggle/working/output
!rm -rf /kaggle/working/skeleton-images.zip
!rm -rf /kaggle/working/yolov8s-pose.pt
!ls -la /kaggle/working/skeleton-images.zip



In [10]:
model = YOLO("yolov8s-pose.pt")

# ================== COCO SKELETON ==================
SKELETON = [
    (0,1),(0,2),(1,3),(2,4),(0,5),(0,6),
    (5,7),(7,9),(6,8),(8,10),(5,6),(5,11),
    (6,12),(11,12),(11,13),(13,15),(12,14),(14,16)
]

# ================== SKELETON VALIDATION ==================
def is_valid_skeleton(conf, th=0.5):
    head = conf[0] > th
    shoulders = conf[5] > th and conf[6] > th
    left_leg = conf[11] > th and conf[13] > th
    right_leg = conf[12] > th and conf[14] > th
    return head and shoulders and (left_leg or right_leg)

# ================== VIDEO ==================
cap = cv2.VideoCapture(VIDEO_PATH)
frame_idx = 0
saved_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1
    if frame_idx % FRAME_SKIP != 0:
        continue

    results = model.track(
        frame,
        imgsz=IMG_SIZE,
        conf=0.4,
        persist=True,
        device="cpu"
    )[0]

    if results.keypoints is None:
        continue

    kpts_xy = results.keypoints.xy.cpu().numpy()
    kpts_conf = results.keypoints.conf.cpu().numpy()
    boxes = results.boxes.xyxy.cpu().numpy()

    for pid, (xy, conf, box) in enumerate(zip(kpts_xy, kpts_conf, boxes)):

        if not is_valid_skeleton(conf, KPT_CONF_TH):
            continue

        x1, y1, x2, y2 = map(int, box)
        h, w = frame.shape[:2]
        x1, y1 = max(0,x1), max(0,y1)
        x2, y2 = min(w,x2), min(h,y2)

        roi_w, roi_h = x2 - x1, y2 - y1
        if roi_w <= 0 or roi_h <= 0:
            continue

        skeleton_roi = np.zeros((roi_h, roi_w, 3), dtype=np.uint8)

        # ---- DRAW JOINTS ----
        for (x, y), c in zip(xy, conf):
            if c < KPT_CONF_TH:
                continue
            cx, cy = int(x - x1), int(y - y1)
            if 0 <= cx < roi_w and 0 <= cy < roi_h:
                cv2.circle(
                    skeleton_roi,
                    (cx, cy),
                    JOINT_RADIUS,
                    (255, 255, 255),
                    -1
                )

        # ---- DRAW BONES (HEAD IS THINNER) ----
        for i, j in SKELETON:
            if conf[i] < KPT_CONF_TH or conf[j] < KPT_CONF_TH:
                continue

            ax, ay = int(xy[i][0] - x1), int(xy[i][1] - y1)
            bx, by = int(xy[j][0] - x1), int(xy[j][1] - y1)

            if not (0 <= ax < roi_w and 0 <= ay < roi_h and
                    0 <= bx < roi_w and 0 <= by < roi_h):
                continue

            # HEAD CONNECTION → thinner line
            thickness = HEAD_BONE_THICKNESS if (i == 0 or j == 0) else BODY_BONE_THICKNESS

            cv2.line(
                skeleton_roi,
                (ax, ay),
                (bx, by),
                (255, 255, 255),
                thickness
            )

        # ---- RESIZE ----
        skeleton_224 = cv2.resize(
            skeleton_roi,
            (ROI_SIZE, ROI_SIZE),
            interpolation=cv2.INTER_NEAREST
        )

        # ---- BINARIZE ----
        gray = cv2.cvtColor(skeleton_224, cv2.COLOR_BGR2GRAY)
        _, skeleton_224 = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

        # ---- DILATION (UNCHANGED) ----
        kernel = np.ones((3,3), np.uint8)
        skeleton_224 = cv2.dilate(skeleton_224, kernel, iterations=1)

        # ---- SAVE ----
        filename = f"f{frame_idx:05d}_p{pid:02d}.png"
        cv2.imwrite(os.path.join(OUTPUT_DIR, filename), skeleton_224)
        saved_count += 1

    print(f"Processed frame {frame_idx}")

cap.release()
print(f"✅ DONE. Saved {saved_count} CLEAN skeleton images.")


0: 384x640 1 person, 175.9ms
Speed: 2.5ms preprocess, 175.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 10

0: 384x640 1 person, 179.2ms
Speed: 1.7ms preprocess, 179.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 20

0: 384x640 1 person, 179.4ms
Speed: 1.7ms preprocess, 179.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 30

0: 384x640 1 person, 173.6ms
Speed: 1.6ms preprocess, 173.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 40

0: 384x640 1 person, 185.7ms
Speed: 1.8ms preprocess, 185.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 50

0: 384x640 1 person, 180.1ms
Speed: 5.2ms preprocess, 180.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 60

0: 384x640 1 person, 179.3ms
Speed: 3.2ms preprocess, 179.3ms inference, 0.9ms postprocess per image at shape (1, 3, 

In [11]:
import shutil

# Zip ONLY your 5484 images (fast - 2-3 minutes)
shutil.make_archive('/kaggle/working/skeleton-images', 'zip', '/kaggle/working/output')

print("✅ Zip complete! Download 'skeleton-images.zip' from Output tab")


✅ Zip complete! Download 'skeleton-images.zip' from Output tab
